# Transfer Learning Task

The goal of this task is to illustrate how pretrained neural networks can be used for soving different classification problems.

## Data

We will use the [Cifar 10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). It consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://github.com/mlcollege/rbi/blob/master/flagship/Deep-Learning-and-Image-Processing/images/cifar10.png?raw=1)



In [1]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(y_train.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
(50000, 32, 32, 3)
(50000, 1)


Prepare data

In [2]:
from keras.utils import to_categorical
import numpy as np

n_classes = 10

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

Import pretrained model

In [22]:
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
#for layer in base_model.layers[:-4]:
#    layer.trainable = False

from tensorflow.keras.applications.resnet50 import ResNet50
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

Add new top layers

In [23]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


Compile the model

In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
model.fit(x_train, y_train,
          batch_size = 300, epochs = 20, verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 77s 308ms/step - accuracy: 0.9587 - loss: 0.1284 - val_accuracy: 0.7717 - val_loss: 1.0038
Epoch 2/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 99ms/step - accuracy: 0.9518 - loss: 0.1493 - val_accuracy: 0.7654 - val_loss: 1.0486
Epoch 3/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 17s 100ms/step - accuracy: 0.9592 - loss: 0.1294 - val_accuracy: 0.7746 - val_loss: 0.9572
Epoch 4/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 98ms/step - accuracy: 0.9731 - loss: 0.0823 - val_accuracy: 0.7331 - val_loss: 1.4675
Epoch 5/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 98ms/step - accuracy: 0.9757 - loss: 0.0731 - val_accuracy: 0.8030 - val_loss: 0.9687
Epoch 6/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - accuracy: 0.9783 - loss: 0.0666 - val_accuracy: 0.7897 - val_loss: 1.0551
Epoch 7/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 100ms/step - accuracy: 0.9770 - loss: 0.0697 - val_accuracy: 0.7948 - val_loss: 1.0224
Epoch 8/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 103ms/step - accuracy: 0.9813 - loss: 0.058

## Evaluate the model

First we need to convert probability vectors to class indices.

In [16]:
y_pred = model.predict(x_test)

print(y_pred.shape)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step
(10000, 10)


In [17]:
import numpy as np

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred_class.shape)

(10000,)


In [18]:
from sklearn import metrics
from sklearn.metrics import accuracy_score


print ("Test accuracy: {:.4f}".format(accuracy_score(y_test_class, y_pred_class)))
print ()
print(metrics.classification_report(y_test_class, y_pred_class, digits=4))

Test accuracy: 0.7926

              precision    recall  f1-score   support

           0     0.8687    0.7870    0.8258      1000
           1     0.8750    0.8890    0.8819      1000
           2     0.6902    0.7820    0.7332      1000
           3     0.6067    0.6850    0.6435      1000
           4     0.8300    0.7080    0.7642      1000
           5     0.7890    0.5760    0.6659      1000
           6     0.7287    0.9160    0.8117      1000
           7     0.8507    0.8490    0.8498      1000
           8     0.9131    0.8620    0.8868      1000
           9     0.8433    0.8720    0.8574      1000

    accuracy                         0.7926     10000
   macro avg     0.7996    0.7926    0.7920     10000
weighted avg     0.7996    0.7926    0.7920     10000



In [19]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))

[[787  16  56  22  13   7  27  11  33  28]
 [  6 889   3   4   3   2  10   4  12  67]
 [ 24   1 782  40  40  10  73  16   6   8]
 [ 12   5  66 685  32  77  78  28   7  10]
 [ 22   3  64  64 708  25  71  37   1   5]
 [  3   5  87 208  24 576  45  42   3   7]
 [  0   1  29  35   8   5 916   2   2   2]
 [  5   5  21  44  19  24  16 849   3  14]
 [ 37  31  15  17   4   0   9   4 862  21]
 [ 10  60  10  10   2   4  12   5  15 872]]


In [ ]:
#resnet

from sklearn import metrics
from sklearn.metrics import accuracy_score


print ("Test accuracy: {:.4f}".format(accuracy_score(y_test_class, y_pred_class)))
print ()
print(metrics.classification_report(y_test_class, y_pred_class, digits=4))


In [25]:
model.fit(x_train, y_train,
          batch_size = 300, epochs = 20, verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 57s 232ms/step - accuracy: 0.1503 - loss: 2.2968 - val_accuracy: 0.2481 - val_loss: 1.8505
Epoch 2/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 22s 133ms/step - accuracy: 0.3241 - loss: 1.6881 - val_accuracy: 0.5114 - val_loss: 1.2898
Epoch 3/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 40s 125ms/step - accuracy: 0.5564 - loss: 1.1736 - val_accuracy: 0.6612 - val_loss: 0.9493
Epoch 4/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 127ms/step - accuracy: 0.6905 - loss: 0.8815 - val_accuracy: 0.7396 - val_loss: 0.7946
Epoch 5/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 126ms/step - accuracy: 0.7704 - loss: 0.6708 - val_accuracy: 0.7639 - val_loss: 0.7050
Epoch 6/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 125ms/step - accuracy: 0.8232 - loss: 0.5290 - val_accuracy: 0.7830 - val_loss: 0.6566
Epoch 7/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 21s 125ms/step - accuracy: 0.8551 - loss: 0.4377 - val_accuracy: 0.8006 - val_loss: 0.6308
Epoch 8/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 41s 126ms/step - accuracy: 0.8815 - loss: 0